## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis = 1, inplace = True)


In [3]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
income_counts = application_df['INCOME_AMT'].value_counts()
income_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
incomes_to_replace = [income_counts[income_counts<1000].index]
# Replace in dataframe
for inc in incomes_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(inc,"1M and Over")

# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M and Over       2790
Name: INCOME_AMT, dtype: int64

In [8]:
ask_counts = application_df['ASK_AMT'].value_counts()
ask_counts

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [9]:
ask_amounts_fewest = (ask_counts[ask_counts< 25000])
ask_amounts_fewest


10478       3
15583       3
63981       3
6725        3
7003        2
           ..
5371754     1
30060       1
43091152    1
18683       1
36500179    1
Name: ASK_AMT, Length: 8746, dtype: int64

In [10]:
ask_amts_to_replace = [ask_amounts_fewest.index]

# Replace in dataframe
for ask in ask_amts_to_replace:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(ask,"Other")
    
# Check to make sure binning was successful
application_df['ASK_AMT'].value_counts()

5000     25398
Other     8901
Name: ASK_AMT, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df['CLASSIFICATION'].value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_greaterThanOne = (class_count[class_count> 1])
class_greaterThanOne


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [13]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = [class_count[class_count<1000].index]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
encoded_df = pd.get_dummies(application_df)
encoded_df

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_1M and Over,INCOME_AMT_25000-99999,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_5000,ASK_AMT_Other
0,1,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
3,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,0,1
4,1,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
34295,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
34296,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
34297,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0


In [15]:
# Split our preprocessed data into our features and target arrays
y = encoded_df['IS_SUCCESSFUL']
X = encoded_df.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    )

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dim=len(X_train_scaled[0])
hidden_node1 = 80
hidden_node2 = 30


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node1, activation="relu", input_dim=input_dim))


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node2, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3840      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,301
Trainable params: 6,301
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
from pandas.core.series import FilePath
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("model_callbacks1/",exist_ok=True)
path="model_callbacks1/weights.{epoch:02d}.hdf5"

cp = ModelCheckpoint(filepath=path, save_weights_only=True, period=5)

In [20]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp])

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5782 - accuracy: 0.7124
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5581 - accuracy: 0.7260
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5544 - accuracy: 0.7280
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5531 - accuracy: 0.7286
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5515 - accuracy: 0.7287
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7276
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5503 - accuracy: 0.7279
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5493 - accuracy: 0.7296
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5485 - accuracy: 0.7301
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5480 - accura

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5563 - accuracy: 0.7284 - 458ms/epoch - 2ms/step
Loss: 0.5563454031944275, Accuracy: 0.728396475315094


In [22]:
# Export our model to HDF5 file
nn.save("Alphabet_Soup2.h5")